In [ ]:
#Online_payment_fraud_detection
#Machine Learning project

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE

# Load the dataset
data = pd.read_csv('onlinefraud .csv')
print('Dataset shape:', data.shape)

# One-Hot Encode 'type'
type_new = pd.get_dummies(data['type'])
data_new = pd.concat([data, type_new], axis=1)

# Separate features (X) and target (y)
X = data_new.drop(['isFraud', 'type', 'nameOrig', 'nameDest'], axis=1)
y = data_new['isFraud']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

# Impute missing values
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# Convert back to DataFrames
X_train = pd.DataFrame(X_train, index=y_train.index)
X_test = pd.DataFrame(X_test, index=y_test.index)

# 🔄 Apply SMOTE (OverSampling)
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Train Random Forest with SMOTE data
model_smote = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=7)
print(f'\n{model_smote} (with SMOTE) : ')
model_smote.fit(X_train_res, y_train_res)

# # Test evaluation
y_preds_smote = model_smote.predict(X_test)
print('SMOTE Validation Accuracy : ', accuracy_score(y_test, y_preds_smote))
print('SMOTE Validation Precision : ', precision_score(y_test, y_preds_smote))
print('SMOTE Validation Recall : ', recall_score(y_test, y_preds_smote))

# Display confusion matrix for SMOTE
cm_smote = confusion_matrix(y_test, y_preds_smote)
disp_smote = ConfusionMatrixDisplay(confusion_matrix=cm_smote)
disp_smote.plot(cmap='Blues')
plt.title('Confusion Matrix - Random Forest (SMOTE)')
plt.show()